In [36]:
import os
api_key = os.environ["MISTRAL_API_KEY"]
print(os.getenv("MISTRAL_API_KEY"))

n8zOo7FKHGqM8Jebj7KKTEYGtDaOwF0X


In [37]:
from langchain_community.document_loaders import TextLoader
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_mistralai.embeddings import MistralAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain


In [38]:
# Load data
loader = TextLoader("/Users/rianrachmanto/miniforge3/project/RAG_Drill_Report/data/well_remark.txt")
docs = loader.load()
# Split text into chunks 
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
# Define the embedding model
embeddings = MistralAIEmbeddings(model="mistral-embed",mistral_api_key=api_key)


/Users/rianrachmanto/miniforge3/envs/mlp/lib/python3.9/site-packages/langchain_mistralai/embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [39]:
# Create the vector store 
vector = FAISS.from_documents(documents, embeddings)
# Define a retriever interface
retriever = vector.as_retriever()

In [40]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [41]:
# Set up the local model:
local_model = "llama3.2"
llm = ChatOllama(model=local_model, num_predict=400,
                 stop=["<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"])

# Set up the RAG chain:
prompt_template = """
<|start_header_id|>user<|end_header_id|>
Answer the following question based only on the provided context".
Question: {question}
Context: {context}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Querying the LLM (oviously to test here you must ask a relevant question of your data)
question = "List date of running tools that encountered an obstruction"
#List date of running tools that encountered an obstruction, and also please include if there is any sand encountered during running
print(question)
print(rag_chain.invoke(question))

List date of running tools that encountered an obstruction
Based on the provided context, the running tools that encountered an obstruction are:

1. MU pulling tool & pull flapper valve (@ 478')
2. MU shifting tool w/ roller stem (@ 6600' with 71* deviation)
3. MU GPWP toolstring (@ 1050', 9000' MD, BHP 1724 psi, 180*F)
4. MU 2-3/8" tbg cutter (@ 3372' with 71* deviation)

These are the instances where the running tools encountered an obstruction or a problem during the drilling process, as indicated by "RIH & toolstring stopped going in hole", "POOH" (indicating that the tool did not find any issues), or other similar phrases.
